- 尝试2600张图片的快速建模，摸清serengeti图形数据简单建模的acc究竟为什么水平

- 均为尺寸很大的彩色图片

### including 3 datasets
- train   70%
- validate  20%
- test   10%

In [ ]:


#  Data prepare: 2 classes [5,21] for quick check 
# VGG data prepare

import pandas as pd
import numpy as np
import csv
import json
import random
import matplotlib.pyplot as plt
import os,sys
import numpy as np
import scipy
from scipy import ndimage
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from PIL import Image
import random
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import os,sys
import numpy as np
import scipy
from scipy import ndimage
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from PIL import Image
import random
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt


train_file = '/root/data/S10.json'
with open(train_file) as data_file:
    train_anns=json.load(data_file)


img_5=[]
for i in train_anns['annotations']:
    if i['category_id'] == 5:
        img_5.append(i['image_id']+'.JPG')
img_5_1w = random.sample(img_5, 1500)

img_21=[]
for i in train_anns['annotations']:
    if i['category_id'] == 21:
        img_21.append(i['image_id']+'.JPG')
img_21_1w = random.sample(img_21,1500)

img_list = img_5_1w+img_21_1w

label_dict = {}
for i in train_anns['annotations']:
    i_new = i['image_id'] + '.JPG'
    if i_new in img_list:
        label_dict[i_new]=i['category_id']
        
df = pd.DataFrame.from_dict(label_dict,orient='index',columns=['label'])
df = df.reset_index().rename(columns={'index':'image_id'})
print(df.head())

df['label'].value_counts().plot.bar()
plt.show()

df.label.unique()

df.loc[df.label == 39, 'label']=21

df.label.unique()

df['label'].value_counts().plot.bar()
plt.show()

In [ ]:
df.label.unique()

In [ ]:
df.shape

为了模仿简书上的VGG迁移学习的code做准备，变成0-1 二分类

In [ ]:
df.loc[df.label == 5, 'label']=0
df.loc[df.label == 21, 'label']=1

总共2600条数据

In [ ]:
df

发现有些图像虽然现在数据集中，但却实际并不存在于图像文件夹中

查找实际不存在的images

In [ ]:
imglist = df['image_id'].tolist()
path = '/root/data/'

count = 0
nonelist = []
for img_name in imglist:
    img_path = path + img_name
    try:
        img = image.load_img(img_path)
        count+=1
    except:
        nonelist.append(img_name)

len(nonelist)        

从上面来看，发现18个不存在的图像

看看是哪些图:

In [ ]:
nonelist

去除这些不存在的图形:

In [ ]:
df1=df[~df.image_id.isin(nonelist)]
len(df1)

从上可见，数据集还剩下2582条观测值

划分数据集：  

70% vs 30%

In [ ]:
train_df, validate0_df = train_test_split(df1, test_size=0.30, random_state=523)
train_df = train_df.reset_index(drop=True)
validate0_df = validate0_df.reset_index(drop=True)

In [ ]:
validate_df, test_df = train_test_split(validate0_df, test_size=0.30, random_state=912)
validate_df = validate_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

In [ ]:
len(train_df)

In [ ]:
len(validate_df)

In [ ]:
len(test_df)

In [ ]:
imglist = train_df['image_id'].tolist()

In [ ]:
X_train = np.empty((len(imglist),227,227,3))

In [ ]:
Y_train = np.empty((len(imglist),1))

In [ ]:
X_train.shape

(2065, 227, 227, 3)

In [ ]:
Y_train.shape

In [2]:
(2065, 1)

(2065, 1)

In [ ]:
path = '/root/data/'

In [ ]:
imglist[:2]

将训练集的像素点转换成像素值：

先不做255标准化

In [ ]:
count=0
nonelist=[]
for img_name in imglist:
    img_path = path + img_name
    try:
        img = image.load_img(img_path, target_size=(227,227))
        X_train[count]=img
        count+=1
    except:
        nonelist.append(img_path)

再看看有没有不存在的图像：

In [ ]:
len(nonelist)

看看数据集转化好了么：

In [ ]:
X_train[:1]

准备Y_train数据集：

In [ ]:
label = train_df['label']
Y_train = [i for i in label]

Y_train是list格式的

In [ ]:
type(Y_train)

查看X_train是什么格式：

发现是numpy.ndarray格式的

In [ ]:
type(X_train)

将Y_train的类型转变成numpy.ndarray格式：

In [ ]:
d=np.array(Y_train)

In [ ]:
d.shape

In [ ]:
Y_train=d.reshape(len(Y_train),1)

In [ ]:
Y_train.shape

In [ ]:
train_df[:2]

In [ ]:
Y_train[:2]

#### 到此，train_df已经变成了X_train, Y_train

###########


#### 接下来，准备validate_df:

In [ ]:
imglist = validate_df['image_id'].tolist()
X_validate = np.empty((len(imglist),227,227,3))
X_validate.shape

(517, 227, 227, 3)

In [ ]:
count=0
nonelist_validate=[]
for img_name in imglist:
    img_path = path + img_name
    try:
        img = image.load_img(img_path, target_size=(227,227))
        X_validate[count]=img
        count+=1
    except:
        nonelist_validate.append(img_path)

In [ ]:
X_validate.shape

(517, 227, 227, 3)

In [ ]:
label = validate_df['label']
Y_validate = [i for i in label]
Y_validate = np.array(Y_validate)
Y_validate = Y_validate.reshape(len(Y_validate),1)
Y_validate.shape

(517, 1)

In [ ]:
validate_df[:2]

In [ ]:
Y_validate[:2]

！！需要将5=斑马,21=母狮子 重新赋值成0和1，这样的话，下面的哑变量的取值就只会从0-1取2个值，而不会有21个哑变量了

In [ ]:
# Y_train[Y_train==5] = 0
# Y_train[Y_train==21] = 1

# Y_validate[Y_validate==5] = 0
# Y_validate[Y_validate==21] = 1

### 准备test_df：

In [ ]:
imglist = test_df['image_id'].tolist()
X_test = np.empty((len(imglist),227,227,3))
X_test.shape

In [ ]:
count=0
nonelist_test=[]
for img_name in imglist:
    img_path = path + img_name
    try:
        img = image.load_img(img_path, target_size=(227,227))
        X_test[count]=img
        count+=1
    except:
        nonelist_test.append(img_path)

In [ ]:
X_test.shape

In [ ]:
label = test_df['label']
Y_test = [i for i in label]
Y_test = np.array(Y_test)
Y_test = Y_test.reshape(len(Y_test),1)
Y_test.shape

验证，看看里面值对的上么：

In [ ]:
test_df[:2]

In [ ]:
Y_test[:2]

====================================

#### Modeling

In [ ]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import keras
from keras.datasets import cifar10
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16
import numpy as np


classes_num = 2
batch_size = 64
epochs_num = 0

In [ ]:
from keras.layers import Flatten,Dense,Dropout,Input
from keras.applications import VGG16
from load_data import load_data_split
from keras.optimizers import SGD
from keras.models import Model 
from keras.utils import np_utils
import config
print('[INFO] loading dataset......')
(x_train,y_train)=(X_train, Y_train)
(x_valid,y_valid)=(X_validate, Y_validate)

y_train=np_utils.to_categorical(y_train,2)
y_valid=np_utils.to_categorical(y_valid,2)
print('[INFO] initializing model......')
base_model=VGG16(weights='imagenet',include_top=False,input_tensor=Input(shape=(227,227,3)))

#微调
head_model=base_model.output
head_model=Flatten(name="flatten")(head_model)
head_model = Dense(512, activation="relu")(head_model)
head_model = Dropout(0.5)(head_model)
head_model=Dense(64,activation='relu')(head_model)
head_model = Dense(len(config.CLASSES), activation="softmax")(head_model)
model=Model(base_model.input,head_model)

#冻结前面的5个卷积组，只训练自定义的全连接层
for layer in base_model.layers:
    layer.trainable=False
print('[INFO] compiling model')
sgd=SGD(lr=0.0001,momentum=0.9)
model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer=sgd)
print('[INFO] training model')
model.fit(x_train, y_train, batch_size=32, epochs=2, validation_data=(x_valid,y_valid))
print('[INFO] saving model and weights')
#保存模型（不含权重）
model_json=model.to_json()
open('model_architecture.json','w').write(model_json)
#保存权重
model.save_weights('transfer_learning_weights.h5', overwrite=True)